In [1]:
import os
import sys
import cv2
import numpy as np
import copy
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from PIL import Image


In [2]:
sys.path.append('..')
from task_2a import detect_ArUco_details, mark_ArUco_image

In [7]:

img = cv2.imread('sample.png')
img2 = cv2.imread('eval.jpg')

img = cv2.resize(img, (960, 960))
img2 = cv2.resize(img2, (960, 957))

orgimg = copy.deepcopy(img)
org2 = copy.deepcopy(img2)
ArUco_details_dict, ArUco_corners = detect_ArUco_details(img)
img = mark_ArUco_image(img, ArUco_details_dict, ArUco_corners) 

ArUco_details_dict, ArUco_corners = detect_ArUco_details(img2)
img2 = mark_ArUco_image(img2, ArUco_details_dict, ArUco_corners) 


# cv2.imshow("Marked Image",img)
cv2.imshow("Marked Image2",img2)

cv2.waitKey(0)
cv2.destroyAllWindows()

Bottom Left Event : 2 C(x-75), 2 C(y+ 75)
Top Left Event : 22 C(y+20), 15 C(y-20)
Mid Left Event : 5 C(x-100), 5 C(y+60)
Top Right Event : 54 C(x+50), 39 C(x-50)
Bottom Right Event : 53 C, 0 C(y+60)

In [22]:
# print(ArUco_corners[55])
marking_img = np.copy(org2)
centers = copy.deepcopy(ArUco_details_dict)
corners = copy.deepcopy(ArUco_corners)
events = [
    [corners[21][0], corners[7][1]], 
    [corners[28][1], corners[14][0]],
    [corners[31][1], corners[11][3]],     
    [corners[34][0], corners[22][1]], 
    [corners[54][2], corners[40][0]]   
]

# changes = (
#     (({0: -72, 1: 9}), ({1: 50, 0: -12 })),
#     (({0: 30, 1: 10}), ({1: 50, 0: -20})), 
#     (({0: 80, 1: 8}), ({0: -118, 1: -18})), 
#     (({0: -135, 1: 3}), ({1: 51, 0: -68})), 
#     (({1: 45, 0: 33}), ({1: -57, 0: -30})),    
# )
# for event in range(5):
#     for corner in range(2):
#         for axe in changes[event][corner]:
#             events[event][corner][axe] += changes[event][corner][axe]

i=0
eventlist=[]
for tl, br in events:
    
    cv2.rectangle(marking_img, tl, br, (0, 255, 0), 2)
    roi = org2[tl[1]:br[1], tl[0]:br[0]]
    print(tl, br, roi)
    eventlist.append(roi)
    i+=1

cv2.imshow("Original Image", marking_img)

cv2.waitKey(0)
cv2.destroyAllWindows()


[317, 793] [68, 902]
[]
[575, 610] [866, 705]
[[[164 154 154]
  [254 249 249]
  [250 243 242]
  ...
  [152 137 128]
  [149 133 126]
  [146 130 123]]

 [[109  90  90]
  [255 247 247]
  [251 244 243]
  ...
  [151 138 128]
  [149 134 127]
  [143 128 122]]

 [[ 93  68  67]
  [255 245 246]
  [253 251 249]
  ...
  [139 129 119]
  [140 126 120]
  [136 122 117]]

 ...

 [[124  84  78]
  [123  83  78]
  [123  83  78]
  ...
  [243 241 233]
  [253 250 245]
  [251 246 244]]

 [[121  81  76]
  [123  80  76]
  [121  81  76]
  ...
  [243 242 234]
  [254 252 248]
  [253 248 247]]

 [[125  82  77]
  [123  79  75]
  [123  80  75]
  ...
  [246 246 239]
  [251 247 244]
  [240 234 233]]]
[603, 439] [856, 536]
[[[ 75  61  61]
  [204 198 196]
  [255 254 250]
  ...
  [106  91  89]
  [104  89  86]
  [104  90  84]]

 [[ 97  76  75]
  [224 214 212]
  [254 251 247]
  ...
  [103  88  87]
  [102  87  85]
  [102  88  85]]

 [[ 96  69  66]
  [219 207 203]
  [253 251 245]
  ...
  [ 99  86  84]
  [ 99  86  84]
  [100  

In [5]:
from matplotlib import cm

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.load("model.tf")
model.eval()
# image = np.expand_dims(image_resized, axis=0)
image_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224), antialias=False),
        
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
print(eventlist)
for image in eventlist:
    # plt.imshow(image[:,:,::-1])
    # plt.show()

    # image = cv2.resize(image, (224, 224))


    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    print(image)
    # path = "EDSR_x4.pb"
    
    # sr.readModel(path)
    
    # sr.setModel("edsr",4)
    
    # result = sr.upsample(image)
    resized = cv2.resize(image,dsize=None,fx=5,fy=5)
    result = image
    
    
    plt.figure(figsize=(12,8))
    plt.subplot(1,3,1)
    # Original image
    plt.imshow(image[:,:,::-1])
    plt.subplot(1,3,2)
    # SR upscaled
    plt.imshow(result[:,:,::-1])
    plt.subplot(1,3,3)
    # OpenCV upscaled
    plt.imshow(resized[:,:,::-1])
    plt.show()
    
    # OpenCV upscaled
    with torch.inference_mode():
        # 6. Transform and add an extra dimension to image (model requires samples in [batch_size, color_channels, height, width])
        transformed_image = image_transform(result).unsqueeze(dim=0)
        # 7. Make a prediction on image with an extra dimension and send it to the target device
        target_image_pred = model(transformed_image.to(device))

    # 8. Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # 9. Convert prediction probabilities -> prediction labels
    pred = torch.argmax(target_image_pred_probs, dim=1)

    # pred = model.predict(image)
    class_names = ['combat', 'destroyedbuilding', 'fire', 'humanitarianaid', 'militaryvehicles']
    event = class_names[pred]
    print(event)

[array([], shape=(109, 0, 3), dtype=uint8), array([[[164, 154, 154],
        [254, 249, 249],
        [250, 243, 242],
        ...,
        [152, 137, 128],
        [149, 133, 126],
        [146, 130, 123]],

       [[109,  90,  90],
        [255, 247, 247],
        [251, 244, 243],
        ...,
        [151, 138, 128],
        [149, 134, 127],
        [143, 128, 122]],

       [[ 93,  68,  67],
        [255, 245, 246],
        [253, 251, 249],
        ...,
        [139, 129, 119],
        [140, 126, 120],
        [136, 122, 117]],

       ...,

       [[124,  84,  78],
        [123,  83,  78],
        [123,  83,  78],
        ...,
        [243, 241, 233],
        [253, 250, 245],
        [251, 246, 244]],

       [[121,  81,  76],
        [123,  80,  76],
        [121,  81,  76],
        ...,
        [243, 242, 234],
        [254, 252, 248],
        [253, 248, 247]],

       [[125,  82,  77],
        [123,  79,  75],
        [123,  80,  75],
        ...,
        [246, 246, 239],
     

error: OpenCV(4.8.0) D:\bld\libopencv_1692668351693\work\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
